<div align="center">

# <span style="color: #3498db;">CA2 - Genetic and Games Algorithm</span>

**<span style="color:rgb(247, 169, 0);">[Mohammad Sadra Abbasi]</span> - <span style="color:rgb(143, 95, 195);">[810101469]</span>**

</div>

# <span style="color: #3498db;">Dots and Boxes with Minimax Algorithm</span>

## Imports

In [1]:
import math
import numpy as np

from tkinter import *
from dataclasses import dataclass
from typing import List, Optional, Tuple
import time
import pandas as pd
from tqdm.notebook import tqdm 

## GameState Class

In [2]:
@dataclass
class GameState:
    board_status: np.ndarray  # shape (n-1, n-1)
    row_status: np.ndarray    # shape (n, n-1)
    col_status: np.ndarray    # shape (n-1, n)
    player1_turn: bool

    def clone(self) -> "GameState":
        """Creates a deep copy of the current game state."""
        return GameState(
            board_status=self.board_status.copy(),
            row_status=self.row_status.copy(),
            col_status=self.col_status.copy(),
            player1_turn=self.player1_turn,
        )

    def get_dimensions(self) -> int:
        """Returns the number of dots (n) in the game grid."""
        return self.row_status.shape[0]

    def is_gameover(self) -> bool:
        """Checks if all edges have been drawn."""
        return (self.row_status == 1).all() and (self.col_status == 1).all()

    def get_scores(self) -> Tuple[int, int]:
        """Returns (player1_score, player2_score)."""
        p1 = int(np.count_nonzero(self.board_status == -4))
        p2 = int(np.count_nonzero(self.board_status == 4))
        return p1, p2

    def available_moves(self) -> List[Tuple[str, List[int]]]:
        """Returns a list of all valid moves as (move_type, [row, col])."""
        moves = []
        for c in range(self.row_status.shape[0]):
            for r in range(self.row_status.shape[1]):
                if self.row_status[c, r] == 0:
                    moves.append(("row", [r, c]))

        for c in range(self.col_status.shape[0]):
            for r in range(self.col_status.shape[1]):
                if self.col_status[c, r] == 0:
                    moves.append(("col", [r, c]))

        return moves

    def apply_move(self, move_type: str, logical_position: List[int]) -> bool:
        """
        Applies a move to the game state.
        Returns True if a box was completed, False otherwise.
        """
        r = logical_position[0]
        c = logical_position[1]
        val = 1
        player_modifier = -1 if self.player1_turn else 1

        scored = False
        n_dots = self.get_dimensions()

        if c < (n_dots - 1) and r < (n_dots - 1):
            cur = abs(self.board_status[c, r]) + val
            self.board_status[c, r] = cur * player_modifier
            if abs(self.board_status[c, r]) == 4:
                scored = True

        if move_type == "row":
            self.row_status[c, r] = 1
            if c >= 1:
                cur = abs(self.board_status[c - 1, r]) + val
                self.board_status[c - 1, r] = cur * player_modifier
                if abs(self.board_status[c - 1, r]) == 4:
                    scored = True

        elif move_type == "col":
            self.col_status[c, r] = 1
            if r >= 1:
                cur = abs(self.board_status[c, r - 1]) + val
                self.board_status[c, r - 1] = cur * player_modifier
                if abs(self.board_status[c, r - 1]) == 4:
                    scored = True

        return scored

## MinimaxAgent Class


In [3]:
class MinimaxAgent:
    def __init__(self, isPlayer1: bool = False, use_pruning: bool = True , heuristic_id: int = 1, debug_tree: bool = False):
        """
        Initialize the Minimax agent.
        Args:
            isPlayer1: True if this agent is Player 1, False if Player 2
        """
        self.node_count = 0
        self.isPlayer1 = isPlayer1
        self.use_pruning = use_pruning
        self.heuristic_id = heuristic_id
        self.debug_tree = debug_tree
    
    def _dbg(self, *args, **kwargs):
        if self.debug_tree:
            print(*args, **kwargs)
    
    @staticmethod
    def heuristic1(state: GameState, agentIsPlayer1: bool) -> float:
        p1, p2 = state.get_scores()
        score = (p1 - p2) if agentIsPlayer1 else (p2 - p1)

        value = score * 200

        board = np.abs(state.board_status)
        threes = np.count_nonzero(board == 3)
        twos   = np.count_nonzero(board == 2)

        maximizing_turn = MinimaxAgent._is_maximizing_turn(state, agentIsPlayer1)

        value -= threes * 150

        if maximizing_turn:
            # value += threes * 150
            value -= twos * 50
        else:
            # value -= threes * 200
            value += twos * 20

        return value



    @staticmethod
    def heuristic2(state: GameState, agentIsPlayer1: bool) -> float:
        p1, p2 = state.get_scores()
        score = (p1 - p2) if agentIsPlayer1 else (p2 - p1)

        value = score * 200

        board = np.abs(state.board_status)
        threes = np.count_nonzero(board == 3)
        twos   = np.count_nonzero(board == 2)

        maximizing_turn = MinimaxAgent._is_maximizing_turn(state, agentIsPlayer1)

        value -= threes * 150

        if maximizing_turn:
            value -= twos * 40
        else:
            value -= threes * 400
            value += twos * 15

        # ----------------------------
        #      Potential Lookahead
        # ----------------------------

        available = state.available_moves()
        n = len(available)

        # potential_weight = min(1.3, 0.25 + (25 - n) * 0.035) <--- it works for 4x4 board size

        dims = state.get_dimensions()
        total_possible_moves = 2 * dims * (dims - 1)
        max_val = total_possible_moves + 1
        potential_weight = min(1.3, 0.25 + (max_val - n) * 0.035)

        potential_sum = 0

        for move_type, pos in available:
            test = state.clone()
            mover_is_me = MinimaxAgent._is_maximizing_turn(test, agentIsPlayer1)

            scored = test.apply_move(move_type, pos)

            after = np.abs(test.board_status)
            new_threes = np.count_nonzero(after == 3) - threes
            new_twos = np.count_nonzero(after == 2) - twos

            move_value = 0

            if mover_is_me:
                if scored:
                    move_value += 220
                    move_value -= new_threes * 120
                    move_value -= new_twos * 40
                else:
                    move_value -= new_threes * 150
                    move_value += new_twos * 15
            else:
                if scored:
                    move_value -= 250
                    move_value -= new_threes * 150
                    move_value += new_twos * 20
                else:
                    move_value += new_threes * 100
                    move_value -= new_twos * 35

            potential_sum += move_value

        if n > 0:
            value += (potential_sum / n) * potential_weight

        return value

    
    def _evaluate(self, state: GameState, agentIsPlayer1: bool) -> float:
        if (self.heuristic_id == 1):
            return MinimaxAgent.heuristic1(state, agentIsPlayer1)
        elif (self.heuristic_id == 2):
            return MinimaxAgent.heuristic2(state, agentIsPlayer1)


    @staticmethod
    def _is_maximizing_turn(state: GameState, agentIsPlayer1: bool) -> bool:
        return state.player1_turn == agentIsPlayer1


    @staticmethod
    def _order_moves(state: GameState, moves: List[Tuple[str, List[int]]]) -> List[Tuple[str, List[int]]]:
        """
        Helper method (PROVIDED): Orders moves for better alpha-beta pruning efficiency.
        Prioritizes moves that complete boxes, then safe moves, then risky moves.
        You can use this in your choose_move() and _minimax() methods.
        """
        completing_moves = []
        safe_moves = []
        risky_moves = []

        for move_type, pos in moves:
            test_state = state.clone()
            scored = test_state.apply_move(move_type, pos)

            if scored:
                completing_moves.append((move_type, pos))
            else:
                boxes_at_3 = int(np.count_nonzero(np.abs(test_state.board_status) == 3))
                if boxes_at_3 > int(np.count_nonzero(np.abs(state.board_status) == 3)):
                    risky_moves.append((move_type, pos))
                else:
                    safe_moves.append((move_type, pos))

        return completing_moves + safe_moves + risky_moves

    def choose_move(
        self, state: GameState, max_depth: Optional[int] = None
    ) -> Tuple[Optional[str], Optional[List[int]], float]:
        # set default max_depth
        if max_depth is None:
            max_depth = 4

        # end game if no moves
        moves = state.available_moves()
        if not moves:
            return None, None, 0.0
        
        self.node_count = 0
    
        ordered_moves = self._order_moves(state, moves)
        best_val = -math.inf
        best_move = None
        
        alpha = -math.inf
        beta = math.inf

        for move_type, pos in ordered_moves:
            next_state = state.clone()
            scored = next_state.apply_move(move_type, pos)
            # print("Scored =", scored)

            if scored:
                val = self._minimax(next_state, 1, max_depth, alpha, beta, True)
            else:
                next_state.player1_turn = not next_state.player1_turn
                val = self._minimax(next_state, 1, max_depth, alpha, beta, False)

            if val > best_val:
                best_val = val
                best_move = (move_type, pos)
            
            alpha = max(alpha, val)
            
        if best_move:
            return best_move[0], best_move[1], best_val
        else:
            return moves[0][0], moves[0][1], best_val

    def _minimax(self, state, depth, max_depth, alpha, beta, maximizing, indent=""):

        self.node_count += 1
        self._dbg(indent, f"Node d={depth} | max={maximizing} | α={alpha} | β={beta}")
    
        if depth == max_depth or state.is_gameover():
            return self._evaluate(state, self.isPlayer1)
    
        moves = state.available_moves()
        ordered_moves = self._order_moves(state, moves)
    
        if maximizing:
            max_eval = -math.inf
            for move_type, pos in ordered_moves:
            
                self._dbg(indent, f"   Try { (move_type, pos) }")
    
                next_state = state.clone()
                scored = next_state.apply_move(move_type, pos)
                # print("Scored =", scored)

    
                if scored:
                    eval = self._minimax(next_state, depth+1, max_depth, alpha, beta, True, indent+"   ")
                else:
                    next_state.player1_turn = not next_state.player1_turn
                    eval = self._minimax(next_state, depth+1, max_depth, alpha, beta, False, indent+"   ")
    
                old_alpha = alpha
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
    
                self._dbg(indent, f"      child={eval} | α:{old_alpha}→{alpha} | β={beta}")
    
                if self.use_pruning and beta <= alpha:
                    self._dbg(indent, f"   ✂ Pruned because β({beta}) ≤ α({alpha})")
                    break
                
            return max_eval
    
        else:
            min_eval = math.inf
            for move_type, pos in ordered_moves:
            
                self._dbg(indent, f"   Try { (move_type, pos) }")
    
                next_state = state.clone()
                scored = next_state.apply_move(move_type, pos)
                # print("Scored =", scored)

    
                if scored:
                    eval = self._minimax(next_state, depth+1, max_depth, alpha, beta, False, indent+"   ")
                else:
                    next_state.player1_turn = not next_state.player1_turn
                    eval = self._minimax(next_state, depth+1, max_depth, alpha, beta, True, indent+"   ")
    
                old_beta = beta
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
    
                self._dbg(indent, f"      child={eval} | α={alpha} | β:{old_beta}→{beta}")
    
                if self.use_pruning and beta <= alpha:
                    self._dbg(indent, f"   ✂ Pruned because β({beta}) ≤ α({alpha})")
                    break
                
            return min_eval
    

## Testing Your Implementation


In [4]:
import random

class RandomAgent:
    def choose_move(self, state: GameState):
        moves = state.available_moves()
        if not moves:
            return None, None, 0
        
        move = random.choice(moves)
        return move[0], move[1], 0
    

In [5]:
# Helper function for creating initial game states (useful for testing)
def make_initial_state(number_of_dots: int, player1_starts: bool = True) -> GameState:
    """
    Creates a new game state with an empty board.
    """
    board_status = np.zeros((number_of_dots - 1, number_of_dots - 1), dtype=int)
    row_status = np.zeros((number_of_dots, number_of_dots - 1), dtype=int)
    col_status = np.zeros((number_of_dots - 1, number_of_dots), dtype=int)
    return GameState(
        board_status=board_status, 
        row_status=row_status, 
        col_status=col_status, 
        player1_turn=player1_starts
    )

In [6]:
def play_simulation(agent_depth, use_pruning, heuristic_id, num_games=20, board_size=3 , debug_tree=False):
    wins = 0
    losses = 0
    ties = 0
    total_nodes = 0
    total_time = 0
    
    minimax_bot = MinimaxAgent(isPlayer1=False, use_pruning=use_pruning, heuristic_id=heuristic_id , debug_tree=debug_tree)
    random_bot = RandomAgent()

    for i in range(num_games):
        initial_state = make_initial_state(board_size, player1_starts=True)
        state = initial_state
        
        game_nodes = 0
        
        while not state.is_gameover():
            if state.player1_turn: 
                m_type, pos, _ = random_bot.choose_move(state)
                
                scored = state.apply_move(m_type, pos)
                
                if not scored:
                     state.player1_turn = False
                     
            else: 
                start_move = time.time()
                m_type, pos, _ = minimax_bot.choose_move(state, max_depth=agent_depth)
                end_move = time.time()
                
                total_time += (end_move - start_move)
                game_nodes += minimax_bot.node_count
                
                scored = state.apply_move(m_type, pos)
                
                if not scored:
                     state.player1_turn = True 
        
        p1, p2 = state.get_scores()
        if p2 > p1: wins += 1
        elif p1 > p2: losses += 1
        else: ties += 1
            
        total_nodes += game_nodes

    if num_games > 0:
        avg_time_per_game = total_time / num_games
        avg_nodes = total_nodes / num_games
        win_rate = (wins / num_games) * 100
    else:
        avg_time_per_game, avg_nodes, win_rate = 0, 0, 0
    
    return {
        "Depth": agent_depth,
        "Pruning": "On" if use_pruning else "Off",
        "Heuristic": f"H{heuristic_id}",
        "Win Rate (%)": win_rate,
        "Avg Nodes": int(avg_nodes),
        "Avg Time (s)": round(avg_time_per_game, 4)
    }

In [8]:
def display_clean_results(df):
    clean_df = df.copy()
    
    clean_df['Win Rate (%)'] = clean_df['Win Rate (%)'].apply(lambda x: f"{x:.1f} %")
    
    clean_df['Avg Time (s)'] = clean_df['Avg Time (s)'].apply(lambda x: f"{x:.4f} s")
    
    clean_df['Avg Nodes'] = clean_df['Avg Nodes'].apply(lambda x: f"{x:,}")
    
    from IPython.display import display
    display(clean_df)


## Symple Runner


In [9]:
results = []

test_depth = 3
board_size = 2
games_per_test = 30

print("Comparing Heuristics...")

print("Running Heuristic 1...")
res_h1 = play_simulation(agent_depth=test_depth, use_pruning=True, heuristic_id=1, num_games=games_per_test, board_size=board_size)
results.append(res_h1)

print("Running Heuristic 2...")
res_h2 = play_simulation(agent_depth=test_depth, use_pruning=True, heuristic_id=2, num_games=games_per_test, board_size=board_size)
results.append(res_h2)

df_compare = pd.DataFrame(results)
display_clean_results(df_compare)

Comparing Heuristics...
Running Heuristic 1...
Running Heuristic 2...


,Depth,Pruning,Heuristic,Win Rate (%),Avg Nodes,Avg Time (s)
0,3,On,H1,100.0 %,12,0.0005 s
1,3,On,H2,100.0 %,12,0.0004 s


## بررسی نتایج به ازای عمق ها و حالت های مختلف

In [ ]:
import time
import pandas as pd
from tqdm import tqdm 

def run_comprehensive_benchmark(num_games=50, board_size=3):    
    heuristics = [1, 2]
    depths = [2, 4, 6]
    # pruning_options = [False, True]
    pruning_options = [True]

    
    results = []
    
    total_iterations = len(heuristics) * len(depths) * len(pruning_options)
    
    print(f"Starting Comprehensive Benchmark...")
    print(f"Settings: Board Size={board_size}x{board_size} | Games per scenario={num_games}")
    print("-" * 60)

    with tqdm(total=total_iterations, desc="Total Progress") as pbar:
        
        for h_id in heuristics:
            for d in depths:
                for prune in pruning_options:
                    current_games = num_games

                    if d >= 6 and not prune:
                        current_games = 2
                    
                    stats = play_simulation(
                        agent_depth=d, 
                        use_pruning=prune, 
                        heuristic_id=h_id, 
                        num_games=current_games, 
                        board_size=board_size
                    )
                    
                    results.append({
                        "Heuristic": f"H{h_id}",
                        "Depth": d,
                        "Pruning": "Yes" if prune else "No",
                        "Win Rate": stats["Win Rate (%)"],
                        "Avg Nodes": stats["Avg Nodes"],
                        "Avg Time": stats["Avg Time (s)"],
                        "Total Games": current_games
                    })
                    
                    pbar.update(1)

    df = pd.DataFrame(results)
    return df

def display_master_table(df):
    clean_df = df.copy()
    try:
        clean_df['Win Rate'] = clean_df['Win Rate'].apply(lambda x: f"{x:.1f}%" if isinstance(x, (int, float)) else x)
        clean_df['Avg Time'] = clean_df['Avg Time'].apply(lambda x: f"{x:.4f}s" if isinstance(x, (int, float)) else x)
        clean_df['Avg Nodes'] = clean_df['Avg Nodes'].apply(lambda x: f"{x:,}" if isinstance(x, (int, float)) else x)
    except:
        pass
        
    clean_df = clean_df.sort_values(by=['Heuristic', 'Depth', 'Pruning'])
    
    from IPython.display import display
    display(clean_df)

final_df = run_comprehensive_benchmark(num_games=10, board_size=4)

print("\nBenchmark Completed!:")
display_master_table(final_df)

<div style="display: flex; gap: 10px;">
  <div style="flex: 1; text-align: center;">
    <h3>3*3 Board Size</h3>
    <img src="dataframe_3*3.png" width="100%">
  </div>
  <div style="flex: 1; text-align: center;">
    <h3>4*4 Board Size</h3>
    <img src="dataframe_4*4.png" width="100%">
  </div>
</div>

---

## Dots and Boxes Game Implementation


In [7]:
# Game Configuration
MAX_AI_DEPTH = 3

size_of_board = 600
number_of_dots = 6
symbol_size = (size_of_board / 3 - size_of_board / 8) / 2
symbol_thickness = 50
dot_color = "#FFFFFF"
player1_color = '#0492CF'
player1_color_light = '#67B0CF'
player2_color = '#EE4035'
player2_color_light = '#EE7E77'
text_color = "#FFFFFF"
dot_width = 0.25*size_of_board/number_of_dots
edge_width = 0.1*size_of_board/number_of_dots
distance_between_dots = size_of_board / (number_of_dots)

In [8]:
class Dots_and_Boxes():
    def __init__(self):
        self.window = Tk()
        self.window.title('Dots_and_Boxes')
        self.canvas = Canvas(self.window, width=size_of_board, height=size_of_board)
        self.canvas.pack()
        self.window.bind('<Button-1>', self.click)
        self.player1_starts = True
        self.ai_agent = MinimaxAgent(isPlayer1=False)
        self.ai_mode = True
        self.ai_max_depth = MAX_AI_DEPTH
        self.refresh_board()
        self.play_again()

    def play_again(self):
        self.refresh_board()
        self.board_status = np.zeros(shape=(number_of_dots - 1, number_of_dots - 1))
        self.row_status = np.zeros(shape=(number_of_dots, number_of_dots - 1))
        self.col_status = np.zeros(shape=(number_of_dots - 1, number_of_dots))
        self.pointsScored = False
        self.player1_starts = not self.player1_starts
        self.player1_turn = self.player1_starts
        self.reset_board = False
        self.turntext_handle = []
        self.already_marked_boxes = []
        self.ai_agent.isPlayer1 = False
        self.display_turn_text()
        self.ai_move_if_needed()

    def mainloop(self):
        self.window.mainloop()

    def is_grid_occupied(self, logical_position, type):
        r = logical_position[0]
        c = logical_position[1]
        occupied = True
        if type == 'row' and self.row_status[c][r] == 0:
            occupied = False
        if type == 'col' and self.col_status[c][r] == 0:
            occupied = False
        return occupied

    def convert_grid_to_logical_position(self, grid_position):
        grid_position = np.array(grid_position)
        position = (grid_position-distance_between_dots/4)//(distance_between_dots/2)

        type = False
        logical_position = []
        if position[1] % 2 == 0 and (position[0] - 1) % 2 == 0:
            r = int((position[0]-1)//2)
            c = int(position[1]//2)
            logical_position = [r, c]
            type = 'row'
        elif position[0] % 2 == 0 and (position[1] - 1) % 2 == 0:
            c = int((position[1] - 1) // 2)
            r = int(position[0] // 2)
            logical_position = [r, c]
            type = 'col'

        return logical_position, type

    def pointScored(self):
        self.pointsScored = True

    def mark_box(self):
        boxes = np.argwhere(self.board_status == -4)
        for box in boxes:
            if tuple(box) not in [tuple(b) for b in self.already_marked_boxes]:
                self.already_marked_boxes.append(list(box))
                color = player1_color_light
                self.shade_box(box, color)

        boxes = np.argwhere(self.board_status == 4)
        for box in boxes:
            if tuple(box) not in [tuple(b) for b in self.already_marked_boxes]:
                self.already_marked_boxes.append(list(box))
                color = player2_color_light
                self.shade_box(box, color)

    def get_game_state_from_gui(self):
        """Helper method to convert GUI state to GameState object."""
        return GameState(
            board_status=self.board_status.copy(),
            row_status=self.row_status.copy(),
            col_status=self.col_status.copy(),
            player1_turn=self.player1_turn
        )

    def is_ai_turn(self):
        return (not self.ai_agent.isPlayer1 and not self.player1_turn) or \
               (self.ai_agent.isPlayer1 and self.player1_turn)

    def ai_move_if_needed(self):
        if self.reset_board or not self.ai_mode or not self.is_ai_turn():
            return

        while True:
            if self.is_gameover():
                break

            state = GameState(
                board_status=self.board_status.copy(),
                row_status=self.row_status.copy(),
                col_status=self.col_status.copy(),
                player1_turn=self.player1_turn
            )

            move_type, pos, _ = self.ai_agent.choose_move(state, max_depth=self.ai_max_depth)
            if move_type is None:
                break

            self.update_board(move_type, pos)
            self.make_edge(move_type, pos)
            self.mark_box()
            self.refresh_board()

            if not self.pointsScored:
                self.player1_turn = not self.player1_turn

            self.pointsScored = False

            if self.is_gameover():
                self.display_gameover()
                break

            if not self.is_ai_turn():
                self.display_turn_text()
                break

    def update_board(self, type, logical_position):
        r = logical_position[0]
        c = logical_position[1]
        val = 1
        playerModifier = -1 if self.player1_turn else 1

        if c < (number_of_dots-1) and r < (number_of_dots-1):
            self.board_status[c][r] = (abs(self.board_status[c][r]) + val) * playerModifier
            if abs(self.board_status[c][r]) == 4:
                self.pointScored()

        if type == 'row':
            self.row_status[c][r] = 1
            if c >= 1:
                self.board_status[c-1][r] = (abs(self.board_status[c-1][r]) + val) * playerModifier
                if abs(self.board_status[c-1][r]) == 4:
                    self.pointScored()

        elif type == 'col':
            self.col_status[c][r] = 1
            if r >= 1:
                self.board_status[c][r-1] = (abs(self.board_status[c][r-1]) + val) * playerModifier
                if abs(self.board_status[c][r-1]) == 4:
                    self.pointScored()

    def is_gameover(self):
        return (self.row_status == 1).all() and (self.col_status == 1).all()

    def make_edge(self, type, logical_position):
        if type == 'row':
            start_x = distance_between_dots/2 + logical_position[0]*distance_between_dots
            end_x = start_x+distance_between_dots
            start_y = distance_between_dots/2 + logical_position[1]*distance_between_dots
            end_y = start_y
        elif type == 'col':
            start_y = distance_between_dots / 2 + logical_position[1] * distance_between_dots
            end_y = start_y + distance_between_dots
            start_x = distance_between_dots / 2 + logical_position[0] * distance_between_dots
            end_x = start_x

        if self.player1_turn:
            color = player1_color
        else:
            color = player2_color
        self.canvas.create_line(start_x, start_y, end_x, end_y, fill=color, width=edge_width)

    def display_gameover(self):
        player1_score = len(np.argwhere(self.board_status == -4))
        player2_score = len(np.argwhere(self.board_status == 4))

        if player1_score > player2_score:
            text = 'Winner: Player 1 '
            color = player1_color
        elif player2_score > player1_score:
            text = 'Winner: Player 2 '
            color = player2_color
        else:
            text = 'Its a tie'
            color = 'gray'

        self.canvas.delete("all")
        self.canvas.create_text(size_of_board / 2, size_of_board / 3, font="cmr 60 bold", fill=color, text=text)

        score_text = 'Scores \n'
        self.canvas.create_text(size_of_board / 2, 5 * size_of_board / 8, font="cmr 40 bold", fill=text_color,
                                text=score_text)

        score_text = 'Player 1 : ' + str(player1_score) + '\n'
        score_text += 'Player 2 : ' + str(player2_score) + '\n'
        self.canvas.create_text(size_of_board / 2, 3 * size_of_board / 4, font="cmr 30 bold", fill=text_color,
                                text=score_text)
        self.reset_board = True

        score_text = 'Click to play again \n'
        self.canvas.create_text(size_of_board / 2, 15 * size_of_board / 16, font="cmr 20 bold", fill="gray",
                                text=score_text)

    def refresh_board(self):
        for i in range(number_of_dots):
            x = i*distance_between_dots+distance_between_dots/2
            self.canvas.create_line(x, distance_between_dots/2, x,
                                    size_of_board-distance_between_dots/2,
                                    fill='gray', dash = (2, 2))
            self.canvas.create_line(distance_between_dots/2, x,
                                    size_of_board-distance_between_dots/2, x,
                                    fill='gray', dash=(2, 2))

        for i in range(number_of_dots):
            for j in range(number_of_dots):
                start_x = i*distance_between_dots+distance_between_dots/2
                end_x = j*distance_between_dots+distance_between_dots/2
                self.canvas.create_oval(start_x-dot_width/2, end_x-dot_width/2, start_x+dot_width/2,
                                        end_x+dot_width/2, fill=dot_color,
                                        outline=dot_color)

    def display_turn_text(self):
        text = 'Next turn: '
        if self.player1_turn:
            text += 'Player1'
            color = player1_color
        else:
            text += 'Player2'
            color = player2_color

        self.canvas.delete(self.turntext_handle)
        self.turntext_handle = self.canvas.create_text(size_of_board - 5*len(text),
                                                       size_of_board-distance_between_dots/8,
                                                       font="cmr 15 bold", text=text, fill=color)

    def shade_box(self, box, color):
        start_x = distance_between_dots / 2 + box[1] * distance_between_dots + edge_width/2
        start_y = distance_between_dots / 2 + box[0] * distance_between_dots + edge_width/2
        end_x = start_x + distance_between_dots - edge_width
        end_y = start_y + distance_between_dots - edge_width
        self.canvas.create_rectangle(start_x, start_y, end_x, end_y, fill=color, outline='')

    def click(self, event):
        if not self.reset_board:
            grid_position = [event.x, event.y]
            logical_positon, valid_input = self.convert_grid_to_logical_position(grid_position)
            if valid_input and not self.is_grid_occupied(logical_positon, valid_input):
                self.update_board(valid_input, logical_positon)
                self.make_edge(valid_input, logical_positon)
                self.mark_box()
                self.refresh_board()

                if not self.pointsScored:
                    self.player1_turn = not self.player1_turn

                self.pointsScored = False

                if self.is_gameover():
                    self.display_gameover()
                else:
                    self.display_turn_text()
                    if self.ai_mode:
                        self.ai_move_if_needed()
        else:
            self.canvas.delete("all")
            self.play_again()
            self.reset_board = False

## Run the Game

In [9]:
# Start the game
game_instance = Dots_and_Boxes()
game_instance.mainloop()

## سوالات

---

<div dir="rtl" style="text-align: justify; line-height: 1.95; font-family: Tahoma;">

<h3 style="margin-top: 20px;">سؤال ۱:</h3>


<div style="display: flex; gap: 15px; margin-bottom: 25px;">
  <div style="flex: 1; text-align: center; padding: 10px; border-radius: 6px;">
    <h3 style="margin-top: 0;">Board Size 3×3</h3>
    <img src="dataframe_3*3.png" width="100%" style="border-radius: 4px;">
  </div>
  <div style="flex: 1; text-align: center; padding: 10px; border-radius: 6px;">
    <h3 style="margin-top: 0;">Board Size 4×4</h3>
    <img src="dataframe_4*4.png" width="100%" style="border-radius: 4px;">
  </div>
</div>


<p>
در عمق‌های کم مانند ۲، الگوریتم تنها بخشی از حالات پیشِ‌رو را بررسی می‌کند و به همین دلیل کیفیت تصمیم‌گیری محدود می‌ماند.  
با افزایش عمق به مقدار ۴، مشاهده می‌شود که الگوریتم دید کامل‌تری نسبت به مسیر بازی پیدا می‌کند و در نتیجه دقت تصمیم‌گیری و نرخ پیروزی به‌طور محسوسی افزایش می‌یابد. همچنین با افزایش عمق دقت و winrate هم بیشتر می شود اما تعداد نود های دیده شده و سربار محاسباتی نیز به طور نمایی و شدید به ویژه بدون استفاده از purning رشد می کند
</p>

<h4 style="margin-top: 18px;">تأثیر عمق بر هزینهٔ محاسباتی</h4>

<p>
با افزایش عمق، تعداد نودهای بررسی‌شده به‌صورت نمایی رشد می‌کند.  
در آزمایش‌ها مشاهده شد که در عمق ۶ (بدون Pruning) برخی حالت‌ها تا صدها هزار نود را شامل شده‌اند و زمان اجرا به چندین ثانیه رسیده است.  
از آن‌جا که این افزایش شدید در زمان اجرا با بهبود کافی در دقت همراه نیست، استفاده از عمق‌های خیلی زیاد مقرون‌به‌صرفه محسوب نمی‌شود.
</p>

<h4 style="margin-top: 18px;">نقش Pruning</h4>

<ul>
    <li>تعداد نودهای بررسی‌شده به‌طور چشمگیری کاهش پیدا می‌کند،</li>
    <li>زمان اجرا کوتاه‌تر می‌شود،</li>
    <li>در عین حال کیفیت تصمیم‌گیری تقریبا حفظ می‌شود.</li>
</ul>

<p>
این کاهش بار محاسباتی به الگوریتم اجازه می‌دهد در عمق‌های مفید مانند ۴ عملکرد بسیار بهتری داشته باشد.
</p>

<h4 style="margin-top: 18px;">انتخاب عمق بهینه</h4>

<p>
با توجه به نتایج، عمق ۴ بهترین نقطهٔ تعادل بین دقت تصمیم‌گیری و هزینهٔ محاسباتی است. در این عمق:
</p>

<ul>
    <li>نرخ پیروزی نسبت به عمق ۲ بهبود قابل توجهی دارد،</li>
    <li>زمان اجرا همچنان در سطح منطقی و قابل استفاده است،</li>
    <li>هزینهٔ محاسباتی به‌مراتب کمتر از عمق ۶ است.</li>
</ul>

<p>
در نتیجه، برای رسیدن به عملکرد پایدار و کارآمد، استفاده از عمق ۴ (ترجیحاً همراه با Pruning) انتخاب بهینه محسوب می‌شود.
</p>

</div>

---

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-family: Tahoma;">

<h3 style="margin-top: 20px;">سؤال ۲:</h3>

<p>
در الگوریتم Minimax همراه با هرس آلفا–بتا، ترتیب بررسی حرکات نقش بسیار مهمی در میزان کارایی هرس دارد.  
هرچه حرکات بهتر زودتر بررسی شوند، احتمال بسته شدن شاخه‌های بی‌فایده بیشتر می‌شود و بخشی از درخت تصمیم هرگز پردازش نمی‌گردد.  
بنابراین الگوریتم زمانی بیشترین کارایی را دارد که اولین حرکت‌های بررسی‌شده، گزینه‌های قوی‌تری باشند و مقدار آلفا یا بتا را سریع‌تر به‌روزرسانی کنند.  
در این حالت، مقادیر حدی زودتر به دست می‌آیند و شاخه‌هایی که نتیجهٔ آن‌ها قطعاً نامطلوب است، سریع‌تر کنار گذاشته می‌شوند.
</p>
</div>

```python
def _order_moves(state: GameState, moves: List[Tuple[str, List[int]]]) -> List[Tuple[str, List[int]]]:
    completing_moves = []
    safe_moves = []
    risky_moves = []

    for move_type, pos in moves:
        test_state = state.clone()
        scored = test_state.apply_move(move_type, pos)

        if scored:
            completing_moves.append((move_type, pos))
        else:
            before = int(np.count_nonzero(np.abs(state.board_status) == 3))
            after  = int(np.count_nonzero(np.abs(test_state.board_status) == 3))

            if after > before:
                risky_moves.append((move_type, pos))
            else:
                safe_moves.append((move_type, pos))

    return completing_moves + safe_moves + risky_moves
```




<div dir="rtl" style="text-align: justify; line-height: 1.9; font-family: Tahoma;">
<h4 style="margin-top: 18px;">تحلیل کد _order_moves</h4>

<p>
این تابع برای ارزیابی اثر هر حرکت، ابتدا یک نسخهٔ کپی از وضعیت بازی می‌سازد و حرکت را روی آن تست می‌کند تا بدون تغییر حالت اصلی بتوان رفتار آن را تشخیص داد.
حرکت‌هایی که منجر به تکمیل جعبه می‌شوند در اولویت اول قرار می‌گیرند، چون این حرکات مستقیم امتیاز تولید می‌کنند و معمولاً بیشترین تأثیر را در نتیجهٔ بازی دارند. پس از آن، حرکت‌های ایمن قرار داده می‌شوند؛ یعنی حرکت‌هایی که وضعیت خطرناک برای خانه‌ها ایجاد نمی‌کنند و در مجموع ریسک پایینی دارند. در نهایت، حرکت‌هایی که باعث نزدیک شدن یک خانه به تکمیل‌شدن می‌شوند و احتمال امتیازدهی حریف را بالا می‌برند، در دستهٔ حرکات پرریسک قرار می‌گیرند.
این ترتیب‌بندی باعث می‌شود Minimax ابتدا حرکات مهم‌تر و کم‌خطرتر را بررسی کند و مقدار آلفا یا بتا سریع‌تر تنظیم شود. در نتیجه بخش زیادی از شاخه‌های بی‌اثر یا نامطلوب زودتر حذف می‌شوند و حجم جستجو کاهش پیدا می‌کند.
</p>

<h4 style="margin-top: 18px;">نسخه بهبود یافته پیشنهادی</h4>

<p>
می توانیم از ایده ای که برای هیوریستیک دوم داشتیم استفاده کنیم یعنی فقط به اثر فوری حرکت نگاه نکنیم، بلکه ببینیم این حرکت در چند قدم بعد چه وضعیتی ایجاد می‌کند. بعضی حرکت‌ها ممکن است در لحظه امتیاز ندهند یا حتی معمولی به‌نظر برسند، اما در ادامه تعداد حرکت‌های امن را بیشتر می‌کنند یا فضای بازی را به نفع ما تغییر می‌دهند. برعکس، بعضی حرکت‌ها در ظاهر بی‌خطرند اما در آینده موقعیت‌های ریسکی زیادی ایجاد می‌کنند.
برای این کار کافی است بعد از تست هر حرکت، وضعیت جدید را بررسی کنیم و حرکت‌های ممکن در آن حالت را به‌طور سطحی امتیازدهی کنیم. اگر حرکت آینده را به سمت موقعیت‌های امن‌تر ببرد امتیاز مثبت می‌گیرد و اگر تعداد موقعیت‌های خطرناک بیشتر شود امتیاز منفی اضافه می‌شود. این امتیاز به امتیاز اصلی حرکت اضافه می‌شود و ترتیب نهایی را دقیق‌تر می‌کند.
</p>
</div>

---

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-family: Tahoma;">

<h3 style="margin-top: 20px;">سؤال ۳:</h3>

<p>
Branching Factor یعنی تعداد حرکت‌هایی که در هر وضعیت از بازی می‌توان انجام داد.  
درواقع نشان می‌دهد از یک حالت، چند مسیر جدید درخت تصمیم باز می‌شود. هرچه این عدد بیشتر باشد، درخت تصمیم بزرگ‌تر می‌شود و Minimax باید حالات بیشتری را بررسی کند.
</p>

<p>
در بازی Dots and Boxes این مقدار در ابتدای بازی بیشترین مقدار خودش را دارد، چون تقریباً همهٔ خطوط خالی هستند.  
مثلاً در صفحهٔ 3×3 (با ۹ نقطه) در مجموع ۱۲ خط ممکن داریم؛ در حرکت اول هر ۱۲ خط قابل انتخاب است.  
اما با جلو رفتن بازی و کشیده شدن خطوط، تعداد حرکت‌های مجاز کم می‌شود و ممکن است در انتهای بازی فقط ۲ یا ۳ حرکت باقی بماند.
این کاهش تدریجی باعث می‌شود Minimax در شروع بازی جستجوی سنگین‌تری داشته باشد،  
ولی در مراحل پایانی که Branching Factor کوچک‌تر است، بتواند عمق بیشتری را با هزینهٔ محاسباتی کمتر بررسی کند.  
</p>

</div>

---

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-family: Tahoma;">

<h3 style="margin-top: 20px;"> سؤال ۴</h3>

<p>
 هرس آلفا-بتا تکنیکی برای بهینه‌سازی الگوریتم Minimax است. هدف اصلی این است که بدون بررسی تمام شاخه‌های درخت بازی، دقیقاً به همان نتیجه‌ای برسیم که الگوریتم کامل Minimax می‌رسد. این یعنی هیچ حرکت بهینه‌ای از دست نمی‌رود، بلکه فقط مسیرهایی که "مطمئن هستیم نتیجه نهایی را تغییر نمی‌دهند" حذف می‌شوند.به این صورت کار می کند که ما دو متغیر را در طول جستجو در درخت همراه خود نگه می‌داریم:آلفا یعنی بهترین (بالاترین) امتیازی که بازیکن Maximizer تا این لحظه در مسیر جستجو تضمین کرده است. (مقدار اولیه: منفی بی نهیات)و بتا یعنی بهترین (پایین‌ترین) امتیازی که بازیکن Minimizer (حریف) تا این لحظه در مسیر جستجو تضمین کرده است. (مقدار اولیه: مثبت بی نهایت) شرط هرس کردن:اگر در یک نود متوجه شویم که بتا کوچتر مساوی آلفا شده جستجو در آن شاخه را متوقف می‌کنیم چون این یعنی حریف مسیری پیدا کرده که امتیازش برای ما کمتر یا مساوی مسیری است که قبلاً پیدا کرده بودیم. پس حریف قطعاً ما را به سمت آن امتیاز پایین هل می‌دهد. چون ما قبلاً راه بهتری (آلفا) سراغ داشتیم، هرگز وارد این شاخه نمی‌شویم. پس دیدن بقیه زیرشاخه‌های آن اتلاف وقت است.
</p>

<h4 style="margin-top: 18px;">مثال برای یک بورد ۲*۲</h4>

<div style="direction:ltr; text-align:left; margin: 15px 0;">
<pre><code>
 Node d=1 | max=False | α=-inf | β=inf
    Try ('row', [0, 1])
    Node d=2 | max=True | α=-inf | β=inf
       Try ('col', [0, 0])
       Node d=3 | max=True | α=-inf | β=inf
          child=50 | α:-inf→50 | β=inf
       child=50 | α=-inf | β:inf→50
    Try ('col', [0, 0])
    Node d=2 | max=True | α=-inf | β=50
       Try ('row', [0, 1])
       Node d=3 | max=True | α=-inf | β=50
          child=50 | α:-inf→50 | β=50
       ✂ Pruned because β(50) ≤ α(50)
       child=50 | α=-inf | β:50→50
 Node d=1 | max=False | α=50 | β=inf
</code></pre>
</div>

<p>
در این قسمت MIN یک فرزند را بررسی می‌کند و مقدار ۵۰ به‌دست می‌آورد.  
بنابراین مقدار <b>بتا از ∞ به ۵۰</b> کاهش پیدا می‌کند.  
اما آلفا هم‌اکنون <b>۵۰</b> است؛ یعنی MIN هیچ شانسی ندارد که مقدار کوچک‌تر از ۵۰ پیدا کند.  
پس ادامهٔ این شاخه کاملاً بی‌فایده است و الگوریتم با شرط <b>β ≤ α</b> آن را قطع می‌کند.
</p>

</div>

---


<div dir="rtl" style="text-align: justify; line-height: 1.9; font-family: Tahoma;">

<h3 style="margin-top: 20px;">سؤال ۵:</h3>

<p>
وقتی حریف حرکاتش را کاملاً تصادفی انتخاب می‌کند، استفاده از Minimax مخصوصاً در عمق‌های بالا عملاً به‌صرفه نیست.  
Minimax برای هر حالت حریف بدترین حالت ممکن را در نظر می‌گیرد و یک جستجوی کامل روی درخت انجام می‌دهد.  
اما اگر حریف اصلاً استراتژیک رفتار نمی‌کند و به شکل تصادفی بازی می‌کند، هزینهٔ محاسباتی Minimax بیش از چیزی است که واقعاً نیاز است.
</p>

<p>
در چنین شرایطی، الگوریتمی مانند  
<b>Monte Carlo Tree Search (MCTS)</b>  
عملکرد بسیار بهتری دارد، چون به‌جای جستجوی کامل و عمیق، با نمونه‌برداری تصادفی مسیرهای مختلف را امتحان می‌کند و میانگین نتایج را مبنای تصمیم‌گیری قرار می‌دهد.
بنابر این اگر حریف غیر قابل پیشبینی باشد یا فضای حالت خیلی بزرگ باشد این الگوریتم بسیار کارآمد است
</p>

<h4 style="margin-top: 18px;">نقش UCB و نحوهٔ کار MCTS</h4>

<p>
MCTS برای انتخاب حرکت از معیار  
<b>Upper Confidence Bound – UCB</b>  
استفاده می‌کند. ایدهٔ اصلی UCB این است که بین دو چیز تعادل برقرار کند:
</p>

<ul>
    <li><b>Exploration</b>: امتحان حرکت‌هایی که کمتر بررسی شده‌اند،</li>
    <li><b>Exploitation</b>: انتخاب حرکت‌هایی که تا اینجا نتیجهٔ خوبی داده‌اند.</li>
</ul>

<p>
UCB برای هر حرکت یک امتیاز می‌سازد که در آن هم «میانگین موفقیت حرکت» و هم «تعداد دفعات امتحان‌شدن آن» در نظر گرفته می‌شود.  
به همین دلیل، MCTS بدون اینکه درخت را کامل بگردد، به مرور زمان بهترین حرکت‌ها را پیدا می‌کند و شاخه‌هایی که امید کمی دارند کمتر مورد بررسی قرار می‌گیرند.
</p>

<h4 style="margin-top: 18px;">مزیت MCTS در برابر Minimax برای حریف تصادفی</h4>

<p>
وقتی حریف تصادفی است، Minimax برای هر حرکت یک درخت عمیق می‌سازد؛ کاری که کاملاً غیرضروری است.  
اما MCTS فقط با شبیه‌سازی‌های زیاد متوجه می‌شود که بسیاری از شاخه‌ها ارزش دنبال‌کردن ندارند و تمرکزش را روی مسیرهای امیدبخش می‌گذارد.  
 به همین دلیل در سناریوهای حریف تصادفی MCTS سریع‌تر بوده و نتیجه بهتری می دهد
</p>

</div>

---